<center>
<h1>Homework #1 - STAT 453</h1>
<h3>By: Jordan Poles</h3>
</center>

In [1]:
from IPython.display import display
import numpy as np
import pandas as pd
import contingency
from contingency import *
from imp import reload
_ = reload(contingency)

## Question 7.2

_Using the data from Table 6.7, estimate the Odds Ratio, with an associated 95%
confidence interval, for the association between current exposure to biomass fuel and
tuberculosis. Repeat your analysis using the small sample adjustments for an estimate and
confidence interval for an Odds Ratio. Compare the results from the two approaches and
comment._

In [2]:
biomass = ContTable(50, 238, 21, 524, 0)
print(biomass.num_tot, "total participants in this study.")

833 total participants in this study.


In [3]:
_ = biomass.odds_ratio(CI=.95, verbose=2)

Odds Ratio (OR): 5.2420968387354945
=> 95% CI using logOR: 1.656722
=> => logOR Var: 0.07372912523788142
=> => Upper Bound: 8.925503208782592
=> => Lower Bound: 3.078770868587116
=> Odds Ratio (OR) indicates a positive relationship between D & E.



In [4]:
_ = biomass.ss_odds_ratio(CI=.95, verbose=2)

Small Sample Odds Ratio (ss_OR): 4.98288322556105
=> 95% CI using ss_logOR: 1.641997
=> => ss_logOR Var: 0.07241305791543796
=> => Upper Bound: 8.753174602530487
=> => Lower Bound: 3.0482772379608516
=> Small Sample Odds Ratio (OR) indicates a positive relationship between D & E.



In [5]:
biomass.OR - biomass.ss_OR

0.25921361317444447

The results of the small sample adjustment for the Odds Ratio (OR) was not largely noticeable. The OR was reduced by the adjustment, by around .26, with a concomitant tiny reduction in the logOR variance of approximately .0013. Given there were 833 participants in this study, the small sample adjustment does not seem particularly necessary in this circumstance.

## Question 7.3

_Tuyns et al. (1977) carried out a case-control study of esophageal cancer in the region
known as Ille-et-Vilaine in Brittany, France. The data set, oesoph, can be found at
http://www.crcpress.eom/ejtroducts/downloads/ and is also examined in detail in Breslow
and Day (1980). One risk factor of interest was daily alcohol consumption, measured in
grams per day, given in the data set in four levels: 0 to 39, 40 to 79, 80 to 120, and >120g/day. Dichotomize this risk factor according to whether an individual’s alcohol.
consumption is less than or greater than or equal to 80 g/day. With this binary measure of alcohol consumption, estimate the Odds Ratio with an associated 95% confidence
interval. Also, examine the relationship between incidence of esophageal cancer and the
dichotomized measure of alcohol consumption using the chisq test._

I sourced my dataset from http://forge.scilab.org/index.php/p/rdataset/source/tree/master/csv/datasets/esoph.csv, as the original link was broken.

In [6]:
esoph = pd.read_csv("data/esoph.csv")
esoph.drop(esoph.columns[[0]], axis=1, inplace=1)
alcbinary = esoph["alcgp"].isin(["80-119", "120+"])
exposed = esoph.loc[alcbinary, ["ncases", "ncontrols"]].sum()
unexposed = esoph.loc[~alcbinary, ["ncases", "ncontrols"]].sum()
esoph_alc_table = ContTable(exposed["ncases"], unexposed["ncases"], exposed["ncontrols"], unexposed["ncontrols"], 0)
esoph_alc_table.display()
esoph_alc_table.odds_ratio(verbose=2)
esoph_alc_table.chisq_indep(verbose=2)

2x2 Contingency Table:


,E,~E
D,96,104
~D,205,770


Odds Ratio (OR): 3.46716697936
=> 95% CI using logOR: 1.243338
=> => logOR Var: 0.0262088013612
=> => Upper Bound: 4.761863084467528
=> => Lower Bound: 2.52448393612794
=> Odds Ratio (OR) indicates a positive relationship between D & E.

ChiSquared Independence Test (n=1175)
=> p-value = 1.66533453694e-15
=> ChiSquared Test contraindicates independence between D & E.



1.6653345369377348e-15

## Question 9.1

_In Perez-Padilla et al. (2001) the authors were concerned that the
variable, monthly family income (an indicator of economic status), might confound the
observed association between indoor air pollution and tuberculosis. The data in Table 6.7,
stratified by income, are shown in Table 9.13, with income information coded as “<1000
pesos per month” and “1000 or more pesos per month.”_

_Based on the income strata, (1) what is the Odds Ratio associating biomass fuel exposure,
for the low income (<1000 pesos/month) stratum? (2) Similarly, what is the Odds Ratio
for the high income (1000 + pesos/month) stratum? Now estimate the Odds Ratio,
associated with biomass fuel exposure, adjusting for income, using the Mantel-Haenszel
method._

### Part 1
We calculate the OR for the individual groups (wealthy and poor) using simple Contingency Tables.

In [7]:
poor_exposure = ContTable(38, 102, 12, 141, 0)
_ = poor_exposure.odds_ratio(verbose=1)

Odds Ratio (OR): 4.377450980392157
=> 95% CI using logOR: 1.476467
=> => logOR Var: 0.1265452429572053
=> => Upper Bound: 8.790648269155707
=> => Lower Bound: 2.1798252528168405


In [8]:
wealthy_exposure = ContTable(12, 136, 9, 383, 0)
_ = wealthy_exposure.odds_ratio(verbose=1)

Odds Ratio (OR): 3.7549019607843137
=> 95% CI using logOR: 1.323062
=> => logOR Var: 0.2044083516783563
=> => Upper Bound: 9.108354899055
=> => Lower Bound: 1.5479511823331233


### Part 2
We now turn to the Mantel-Haenszel method to correct for the effect of this confounding variable on the Odds Ratio.

In [9]:
biofuel_strata = Strata([poor_exposure, wealthy_exposure])
_ = biofuel_strata.mantel_haenszel_correction()

Mantel-Haenszel adj. OR: 4.15847515476779
=> 95% CI using logOR: 1.425148
=> => logOR Var: 0.07841897932703254
=> => Upper Bound: 7.199451163453156
=> => Lower Bound: 2.401976931325776


### Part 3 - Discussion

_Based on your calculations and those reported from Question 6.4, what is your
assessment of confounding by monthly income? Does the income variable fulfill the
criteria required for a variable to be a confounder?
In addition to income, the authors considered many other confounding factors: age,
sex, urban or rural residence, smoking, crowding, level of education, and socioeconomic
status. After controlling for all these confounders, their estimate of the adjusted Odds
Ratio between indoor air pollution and tuberculosis was 2.2. Based on the crude and
adjusted Odds Ratio, was the association of Question 6.4 confounded?_

In [10]:
sixptfour = ContTable(50, 238, 21, 524, 0)
sixptfour.display()
_ = sixptfour.odds_ratio(verbose=1, CI=0)

2x2 Contingency Table:


,E,~E
D,50,238
~D,21,524


Odds Ratio (OR): 5.2420968387354945


Given the above calculations it seems quite clear that income/wealth is an important confounding factor mediating the occurence of tuberculosis in populations exposed to biofuel emissions. 

## Question 9.2

_Refer again to the data set esoph. The data, associating the binary measure of alcohol
consumption with esophageal cancer incidence, can be stratified into
two age groups, 25 to 54 years old and 55 years old and above. Use the Cochran-Mantel-
Haenszel method to examine the association between alcohol consumption and incidence
of esophageal cancer, adjusting for this dichotomous measure of age._

_Give a summary estimate of the Odds Ratio using both the Woolf and Mantel-Haenszel methods. Provide
two 95% confidence intervals based on your summary estimates. Compare these
confidence intervals and comment._

In [22]:
esoph.loc[np.logical_and(alcbinary, agebinary), ["ncases", "ncontrols"]].sum()

ncases        66
ncontrols    111
dtype: int64

In [20]:
agebinary = esoph["agegp"].isin(["55-64", "65-74", "75+"])
#old fok
exposed = esoph.loc[np.logical_and(alcbinary, agebinary), ["ncases", "ncontrols"]].sum()
unexposed = esoph.loc[np.logical_and(~alcbinary, agebinary), ["ncases", "ncontrols"]].sum()
esoph_old = ContTable(exposed["ncases"], unexposed["ncases"], exposed["ncontrols"], unexposed["ncontrols"], 0)
#young folk
exposed = esoph.loc[np.logical_and(alcbinary, ~agebinary), ["ncases", "ncontrols"]].sum()
unexposed = esoph.loc[np.logical_and(~alcbinary, ~agebinary), ["ncases", "ncontrols"]].sum()
esoph_young = ContTable(exposed["ncases"], unexposed["ncases"], exposed["ncontrols"], unexposed["ncontrols"], 0)
print("Elderly Patients:")
esoph_young.display()
esoph_young.odds_ratio(verbose=2)
esoph_young.chisq_indep(verbose=2)
print("Young Patients:")
esoph_young.display()
esoph_young.odds_ratio(verbose=2)
esoph_young.chisq_indep(verbose=2)

Elderly Patients:
2x2 Contingency Table:


,E,~E
D,30,26
~D,94,434


Odds Ratio (OR): 5.32733224223
=> 95% CI using logOR: 1.672851
=> => logOR Var: 0.0847373171327
=> => Upper Bound: 9.425226256467715
=> => Lower Bound: 3.011118040756225
=> Odds Ratio (OR) indicates a positive relationship between D & E.

ChiSquared Independence Test (n=584)
=> p-value = 4.86520823628e-10
=> ChiSquared Test contraindicates independence between D & E.

Young Patients:
2x2 Contingency Table:


,E,~E
D,30,26
~D,94,434


Odds Ratio (OR): 5.32733224223
=> 95% CI using logOR: 1.672851
=> => logOR Var: 0.0847373171327
=> => Upper Bound: 9.425226256467715
=> => Lower Bound: 3.011118040756225
=> Odds Ratio (OR) indicates a positive relationship between D & E.

ChiSquared Independence Test (n=584)
=> p-value = 4.86520823628e-10
=> ChiSquared Test contraindicates independence between D & E.



4.8652082362821147e-10

In [21]:
esoph_strata = Strata([esoph_young, esoph_old])
_ = esoph_strata.mantel_haenszel_correction()

Mantel-Haenszel adj. OR: 3.17591647392
=> 95% CI using logOR: 1.155596
=> => logOR Var: 0.027113562572
=> => Upper Bound: 4.3856057779669735
=> => Lower Bound: 2.299897884113105


In [14]:
_ = esoph_strata.woolf_correction([esoph_young, esoph_old])

Individual ORs: [1.6539424280350439, 3.8338039532069383]


TypeError: unorderable types: ContTable() > int()

In [ ]:
def cmh_test(verbose=1):
    pass